(comp/00-algorithms)=
# Coding Problems

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=brightgreen)
[![Source](https://img.shields.io/static/v1.svg?label=GitHub&message=Source&color=181717&logo=GitHub)](https://github.com/particle1331/ok-transformer/blob/master/docs/nb/comp/00-algorithms.ipynb)
[![Stars](https://img.shields.io/github/stars/particle1331/ok-transformer?style=social)](https://github.com/particle1331/ok-transformer)

---

**References:** {cite}`durr2020competitive` {cite}`Laaksonen20`

## Backtracking

### Reverse Shuffle Merge

**Problem.** [[Reverse Shuffle Merge](https://www.hackerrank.com/challenges/reverse-shuffle-merge/problem) (HR)]. Given a string $S$ containing characters with even counts, we want to find a lexically minimal string $A$ containing half of each characters of $S$ such that $a_i = s_{j_i}$ where $j_{k} \leq j_{k-1} \leq \ldots \leq j_1$ with $k = \frac{1}{2}|S|.$ For example, if $S$ is `eggcegcg` then $A$ is the string `cegg` which can be constructed as `·gg·e·c·` within $S$.

**Solution.** Note that we can iterate over $S$ from right to left choosing to skip or take a character. Since $A$ has to be minimal, we want to take minimal characters, and skip those that are large. However, skipping too often can also result in a suboptimal solution. Consider `S = 'cdeedcaaaa'` here skipping `c` and `d` gets us in trouble since `e` reaches its skip limit of 1/2 its count, and we are forced to take it, getting the suboptimal solution `A = 'aaedc'`. Hence, we want to take characters, unless we are forced to skip, or we have better options:

In [1]:
from collections import Counter

def solve(S):
    counts = Counter(S)
    chars = counts.keys()
    A = ['.']
    take_count = {c: 0 for c in chars}
    skip_count = {c: 0 for c in chars}

    def take(s):
        A.append(s)
        take_count[s] += 1

    def skip(s):
        skip_count[s] += 1

    def squash(s):
        while (len(A) > 1) and (A[-1] > s) and (skip_count[A[-1]] < counts[A[-1]] / 2):
            skip_count[A[-1]] += 1
            take_count[A[-1]] -= 1
            A.pop()
        take(s)

    for s in S[::-1]:        
        if take_count[s] == counts[s] / 2:
            skip(s)
        else:
            squash(s)

    return "".join(A[1:])


S = "eggcegcg"
A = solve(S)
A

'cegg'

Notice that instead of `take(s)` we have `squash(s)`. The trick is to construct chains: `a <= a <= c <= d` and  if we encounter `b` such that `d > b`, then we backtrack by popping `d` and `c` assuming the skip limit allows it. Our sequence then becomes `a <= a <= b`. The equals is important here to allow consecutive same characters. This is also why we added `.` at the start which is smaller than any character. 

Observe that the `squash` function only takes into account `A` and ignores skipped characters. This is fine since instances of these characters are already optimally placed in the chain. It only adds more skipped characters which are suboptimal in the chain compared to the current queried character. Trying a more complicated input:

In [2]:
def viz(S):
    chars = Counter(S)
    d = {c: [] for c in sorted(chars.keys(), reverse=True)}

    for s in S[::-1]:
        for c in d.keys():
            if c == s:
                d[c].append(s + ' ')
            else:
                d[c].append('  ')

    for c in d.keys():
        print(''.join(d[c]))



S = "dcadeebcccaaba."
viz(S)
print()
print(f"S = {S[:-1]}")
print("A =", solve(S)[1:])

                  e e         
                      d     d 
          c c c           c   
    b           b             
  a   a a               a     
.                             

S = dcadeebcccaaba
A = aacbecd


```{figure} ../../img/comp/greedy-1-soln-1.png
---
width: 60%
---
To respect optimality, we try as much as possible to have our curve in the bottom half of the diagonal. At the start, `b` is squashed after getting `a <= b ? a` while the third `a` skipped. Then, we get consecutive `c <= c ? b` (last `c` is skipped due to take limit) only the latter `c` is popped due to skip limit. Next, we encounter two `e`, the latter skipped due to take limit. And so on. Marked characters are skipped while unmarked characters were squashed (essentially also skipped).
```

### Generate Parenthesis

**Problem.** [[Generate Parenthesis](https://leetcode.com/problems/generate-parentheses/description/?envType=study-plan-v2&envId=top-100-liked) (LC)]. Given $n$ pairs of parentheses, generate all strings of well-formed parentheses. For $n = 1$, the only solution is `'()'`.

**Solution.** Suppose you have a string of well-formed parentheses, then you can only add `(`. If you have `(` in excess, you can add enough `)` or add more `(` until its count reaches $n$. Hence, we can just branch off at each candidate solution by adding `(` or `)` while keeping constraints in check. This ultimately results in well-formed parentheses at the leaves. 

In [25]:
def solve(n):
    out = [(0, 0, "")]
    while True:
        tmp = []
        for res in out:
            l, r, s = res
            if l < n:
                tmp.append((l + 1, r, s + "("))
            if l > r:
                tmp.append((l, r + 1, s + ")"))
        
        if len(tmp) == 0:
            return [s for (l, r, s) in out]
        else:
            out = tmp


print(solve(1))
print(solve(3))

['()']
['((()))', '(()())', '(())()', '()(())', '()()()']


**Remark.** Candidate solutions are represented as tuples `(l, r, s)` containing left parenthesis count `l`, right parenthesis count `r`, and the constucted string `s`.

### Combination Sum

**Problem.** [[Combination Sum](https://leetcode.com/problems/combination-sum/?envType=study-plan-v2&envId=top-100-liked) (LC)]. Given a sequence $S = (s_1, \ldots, s_n)$ of distinct integers and a target $t$, generate all sequences $(s_{j_1}, \ldots, s_{j_k})$ such that $s_{j_1} \leq \ldots \leq s_{j_k}$ and $s_{j_1} + \ldots + s_{j_k} = t$. For example, let $S = (2, 3, 5)$ and $t = 8.$ Then, the expected solution is composed of $(2, 2, 2, 2)$, $(2, 3, 3)$, and $(3, 5).$

**Solution.** Our solution iteratively constructs paths (sequences starting from single integers) from previous paths which are still open, i.e. those whose sum does not exceed or equal the target. This would make more sense by looking at the algorithm for the given example:

```{margin}
- `?` = open
- `x` = exceeds target
- `/` = equals target
```
```
depth = 1
5 ?
3 ?
2 ?

depth = 2
5 5 x
5 3 /
5 2 ?
3 3 ?
3 2 ?
2 2 ?

depth = 3
5 2 2 x
3 3 3 x
3 3 2 x
3 2 2 ?
2 2 2 ?

depth = 4
3 2 2 2 x
2 2 2 2 /
```

Note that each open path is appended by an integer less than or equal the last integer. This is important for constructing all unique sequences. The algorithm terminates when there are no more open paths. Below we also cache intermediate path values (i.e. its total sum) for faster evaluation (summing two numbers).

In [80]:
def solve(S, t, reverse=True):
    solution = [[t]] if t in S else []
    S = list(filter(lambda s: s < t, sorted(S, reverse=reverse)))

    paths = list(zip([(i,) for i in range(len(S))], S))

    while len(paths) > 0:
        tmp = []
        for path in paths:
            seq, val = path
            i = seq[-1]
            for j in range(i, len(S)):
                seqj, valj = seq + (j,), val + S[j]
                if valj == t:
                    solution.append([S[k] for k in seqj])
                if valj < t:
                    tmp.append((seqj, valj))
        paths = tmp

    return solution


solve(S=[2, 3, 5, 8], t=8)

[[8], [5, 3], [3, 3, 2], [2, 2, 2, 2]]

**Remark.** It is important to close out larger sequences first. This explains why we reversed `S`. This way we avoid creating long sequences which will exceed the target anyway, e.g. we close out `5 2 2 x` early instead of wasting constructing `2 2 2 5 x` if we start with `2` instead of `5`.

In [81]:
%%timeit
solve(S=[24, 16, 30, 7, 5, 4, 9, 29, 8, 35, 3, 17], t=29);

219 µs ± 3.61 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [82]:
%%timeit
solve(S=[24, 16, 30, 7, 5, 4, 9, 29, 8, 35, 3, 17], t=29, reverse=False);

398 µs ± 9.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


<br>

```{figure} ../../img/comp/combination-sum.png
---
width: 60%
---
This solution is super efficient.
```